<a href="https://colab.research.google.com/github/andrejfg/Metaforando-Webcam/blob/main/Metaforando_WebCam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metaforando - WebCam
Essa aplicação utiliza modelos de classificação de emoções gerado pelo keras. A classificação do modelo é apresentada em tempo real de acordo com a imagem recebida pela webcam em tempo real.
Para isto, o algoritmo precisa reconhecer cada rosto e classificar a expressão facial de acordo com o modelo.

Essa aplicação utiliza dois modelos de reconhecimentos faciais:
* OpenCV Haar Cascades  - https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html
* Face Recognition - https://pypi.org/project/face-recognition/

O modelo da biblioteca Face Recognition possui uma acurácia de 99.38% para reconhecimento de faces, mas é necessário a execução em um ambiente com GPU CUDA. Para ambientes que não possuem GPU CUDA, será utilizado o modelo de reconhecimento de rosto do OpenCV Haar Cascades, que mostrou resultados satisfatórios.  

# Configurações de execução

É definido o caminho onde é encontrado o modelo treinado pelo keras e salvo no Google Drive.

Também deve ser definido o tamanho da entrada da imagem que o modelo foi treinado.

Em seguida é definido os rótulos de treinamento do modelo a ser carregado.

In [ ]:
path_of_model = PATH_OF_MODEL
input_shape = (48,48)
emocoes =[
           'Raiva', 
           'Nojo',
           'Medo', 
           'Alegria', 
           'Tristeza', 
           'Surpresa', 
           'Neutro'
          ]

## Importações

In [ ]:
from keras import models
from IPython.display import Image
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import PIL
import io
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from base64 import b64decode, b64encode

## Verificação de GPU no ambiente de execução

In [ ]:
from tensorflow.config import list_physical_devices
if len(list_physical_devices('GPU')) > 0:
  face_model = "GPU"
  !pip install face_recognition
  import face_recognition
else:
  face_model = "CPU"
  face_recognition = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))



## Importação do modelo no Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Carregamento do modelo

In [ ]:
modelo = models.load_model(path_of_model)

## Funções auxiliares

js_to_image- Captura o frame da imagens e converte para uma imagem do opencv

bbox_to_bytes- Converte a imagem gerada do openCV (caixa de reconhecimento do rosto, Rótulo da emoção) em sring para que seja projetado junto a imagem da webcam

In [ ]:
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

## Função principal para visualização da webcam

In [ ]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'Sem imagem';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'Clique na imagem para encerrar a execução</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

## Função para classificação de emoções com modelo keras

Além da classificação da emoção, também reconhece a maior probabilidade além de todas as outras probabilidades.

In [ ]:
def preprocessa_imagem(imagem, modelo):
  imagem = imagem.astype(np.float64)/255
  imagem = np.expand_dims(imagem, axis = 0)
  probabilidades = modelo.predict(imagem)
  probabilidades = probabilidades[0]
  classificacao = np.argmax(probabilidades)
  certeza = round(probabilidades[np.argmax(probabilidades)]*100, ndigits= 2)

  return classificacao, certeza, probabilidades

A função de execução da aplicação apresenta cada frame e adiciona as caixas e rótulos quando necessário.

In [ ]:
def executa_reconhecimento(face_recognition = face_recognition, face_model = face_model, input_shape = input_shape):
  video_stream()
  # label for video
  label_html = 'Capturando...'
  # initialze bounding box to empty
  bbox = ''

  font = cv2.FONT_HERSHEY_DUPLEX

  while True:
      js_reply = video_frame(label_html, bbox)
      if not js_reply:
          break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      # create transparent overlay for bounding box
      bbox_array = np.zeros([480,640,4], dtype=np.uint8)

      # grayscale image for face detection
      gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

      # get face region coordinates
      if face_model == "GPU":
        faces = face_recognition.face_locations(gray)
        for (top, right, bottom, left) in faces:
          try:
            img_predict = cv2.resize(gray[top:bottom, left:right],input_shape)
            resultado = preprocessa_imagem(img_predict,modelo)
            bbox_array = cv2.rectangle(bbox_array, (left-50, top-50), (right+50, bottom+50), (255, 0, 0), 2)
            bbox_array = cv2.rectangle(bbox_array,(left-50, bottom + 15 ), (right+50, bottom+50), (255, 0, 0), cv2.FILLED)
            bbox_array = cv2.putText(bbox_array,emocoes[resultado[0]], (left, bottom +40), font, 1.0, (255, 255, 255), 1)
          except:
            break
      else:
        faces = face_recognition.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
          try:
            img_predict = cv2.resize(gray[y+25:y+h+25, x-25:x+w+25],(48,48))
            resultado = preprocessa_imagem(img_predict,modelo)
            bbox_array = cv2.rectangle(bbox_array, (x-25,y-25),(x+w+25,y+h+25), (255, 0, 0), 2)
            bbox_array = cv2.rectangle(bbox_array,(x-25,y+h),(x+w+25,y+h+40), (255, 0, 0), cv2.FILLED)
            bbox_array = cv2.putText(bbox_array,emocoes[resultado[0]], (x,y+h+30), font, 1.0, (255, 255, 255), 1)
          except:
            break
  
      # get face bounding box for overlay
        
      bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
      # convert overlay of bbox into bytes
      bbox_bytes = bbox_to_bytes(bbox_array)
      # update bbox so next frame gets new overlay
      bbox = bbox_bytes

# Aplicação

In [ ]:
executa_reconhecimento()

<IPython.core.display.Javascript object>